In [63]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import os

In [64]:
urls = {
    'Anxiety': 'https://medium.com/tag/anxiety/archive/{0}/{1:02d}/{2:02d}',
    'AnxietyMonths' : 'https://medium.com/tag/anxiety/archive/{0}/{1:02d}'
}

In [65]:
def is_leap(year):
    if year % 4 != 0:
        return False
    elif year % 100 != 0:
        return True
    elif year % 400 != 0:
        return False
    else:
        return True
    
def convert_day(day, year):
    month_days = [31, 29 if is_leap(year) else 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    m = 0
    d = 0
    while day > 0:
        d = day
        day -= month_days[m]
        m += 1
    return (m, d)

In [94]:
year = 2016
selected_days = [i for i in range(1, 367 if is_leap(year) else 366)]

In [95]:
img_dir = 'images'
if not os.path.exists(img_dir):
    os.mkdir(img_dir)

In [96]:
data = []
article_id = 0
i = 0
n = len(selected_days)
if int(year) <= 2015:
    url = 'https://medium.com/tag/anxiety/archive/{0}/{1:02d}'
    for month in range(1,13):
        date = '{0}-{1:02d}'.format(year, month)
        print(url.format(year, month))
        response = requests.get(url.format(year, month), allow_redirects=True)
        if response.url.startswith(url.format(year, month)):
            page = response.content
            soup = BeautifulSoup(page, 'html.parser')
            articles = soup.find_all("div", class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")
            for article in articles:
                title = article.find("h3", class_="graf--title")
                if title is None:
                    continue
                title = title.contents[0]
                article_id += 1
                article_url = article.find_all("a")[3]['href'].split('?')[0]

                data.append([article_id, article_url, title, date])
else:
    url = 'https://medium.com/tag/anxiety/archive/{0}/{1:02d}/{2:02d}'
    for d in selected_days:
        i += 1
        month, day = convert_day(d, year)
        date = '{0}-{1:02d}-{2:02d}'.format(year, month, day)
        print(url.format(year, month, day))
        response = requests.get(url.format(year, month, day), allow_redirects=True)
        if not response.url.startswith(url.format(year, month, day)):
            continue
        page = response.content
        soup = BeautifulSoup(page, 'html.parser')
        articles = soup.find_all("div", class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")
        for article in articles:
            title = article.find("h3", class_="graf--title")
            if title is None:
                continue
            #title = BeautifulSoup(title.contents[0])
            only_title = title.get_text()
            article_id += 1
            article_url = article.find_all("a")[3]['href'].split('?')[0]

            data.append([article_id, article_url, only_title, date])
        

https://medium.com/tag/anxiety/archive/2016/01/01
https://medium.com/tag/anxiety/archive/2016/01/02
https://medium.com/tag/anxiety/archive/2016/01/03
https://medium.com/tag/anxiety/archive/2016/01/04
https://medium.com/tag/anxiety/archive/2016/01/05
https://medium.com/tag/anxiety/archive/2016/01/06
https://medium.com/tag/anxiety/archive/2016/01/07
https://medium.com/tag/anxiety/archive/2016/01/08
https://medium.com/tag/anxiety/archive/2016/01/09
https://medium.com/tag/anxiety/archive/2016/01/10
https://medium.com/tag/anxiety/archive/2016/01/11
https://medium.com/tag/anxiety/archive/2016/01/12
https://medium.com/tag/anxiety/archive/2016/01/13
https://medium.com/tag/anxiety/archive/2016/01/14
https://medium.com/tag/anxiety/archive/2016/01/15
https://medium.com/tag/anxiety/archive/2016/01/16
https://medium.com/tag/anxiety/archive/2016/01/17
https://medium.com/tag/anxiety/archive/2016/01/18
https://medium.com/tag/anxiety/archive/2016/01/19
https://medium.com/tag/anxiety/archive/2016/01/20


https://medium.com/tag/anxiety/archive/2016/06/13
https://medium.com/tag/anxiety/archive/2016/06/14
https://medium.com/tag/anxiety/archive/2016/06/15
https://medium.com/tag/anxiety/archive/2016/06/16
https://medium.com/tag/anxiety/archive/2016/06/17
https://medium.com/tag/anxiety/archive/2016/06/18
https://medium.com/tag/anxiety/archive/2016/06/19
https://medium.com/tag/anxiety/archive/2016/06/20
https://medium.com/tag/anxiety/archive/2016/06/21
https://medium.com/tag/anxiety/archive/2016/06/22
https://medium.com/tag/anxiety/archive/2016/06/23
https://medium.com/tag/anxiety/archive/2016/06/24
https://medium.com/tag/anxiety/archive/2016/06/25
https://medium.com/tag/anxiety/archive/2016/06/26
https://medium.com/tag/anxiety/archive/2016/06/27
https://medium.com/tag/anxiety/archive/2016/06/28
https://medium.com/tag/anxiety/archive/2016/06/29
https://medium.com/tag/anxiety/archive/2016/06/30
https://medium.com/tag/anxiety/archive/2016/07/01
https://medium.com/tag/anxiety/archive/2016/07/02


https://medium.com/tag/anxiety/archive/2016/11/24
https://medium.com/tag/anxiety/archive/2016/11/25
https://medium.com/tag/anxiety/archive/2016/11/26
https://medium.com/tag/anxiety/archive/2016/11/27
https://medium.com/tag/anxiety/archive/2016/11/28
https://medium.com/tag/anxiety/archive/2016/11/29
https://medium.com/tag/anxiety/archive/2016/11/30
https://medium.com/tag/anxiety/archive/2016/12/01
https://medium.com/tag/anxiety/archive/2016/12/02
https://medium.com/tag/anxiety/archive/2016/12/03
https://medium.com/tag/anxiety/archive/2016/12/04
https://medium.com/tag/anxiety/archive/2016/12/05
https://medium.com/tag/anxiety/archive/2016/12/06
https://medium.com/tag/anxiety/archive/2016/12/07
https://medium.com/tag/anxiety/archive/2016/12/08
https://medium.com/tag/anxiety/archive/2016/12/09
https://medium.com/tag/anxiety/archive/2016/12/10
https://medium.com/tag/anxiety/archive/2016/12/11
https://medium.com/tag/anxiety/archive/2016/12/12
https://medium.com/tag/anxiety/archive/2016/12/13


In [97]:
for i in range(len(data)):
    url = data[i][1]
    print(url)
    try:
        req = requests.get(url)
        page = requests.get(url)
        soup = BeautifulSoup(page.text, 'html.parser')

        sections = soup.find_all('section')
        all_paragraphs = []
        section_titles = []

        for section in sections:
            paragraphs = section.find_all('p')
            for paragraph in paragraphs:
                all_paragraphs.append(paragraph.text)

            subs = section.find_all('h1')
            for sub in subs:
                only_text = BeautifulSoup(sub.text)
                section_titles.append(only_text.get_text())
        # to ensure that we only append the text, some articles contain mins to read variable
        final_paragraphs = ""
        sep = " "
        if len(all_paragraphs) > 2:
            if "min read" in all_paragraphs[0] or "min read" in all_paragraphs[1]:
                final_paragraphs = sep.join(all_paragraphs[2:])
            else:
                final_paragraphs = sep.join(all_paragraphs[1:])
        else:
            final_paragraphs = sep.join(all_paragraphs)

        data[i].append(final_paragraphs)
    except requests.exceptions.ConnectionError as e:
        data[i].append("999999")
    
    
print(data[0])
print(data[17])

https://medium.com/@elissaerin/27-minutes-of-uber-induced-anxiety-efcf6fdf8ba4
https://medium.com/@AliciaEdquist/thank-you-anxiety-fecfefd3e88
https://medium.com/@guiltysquid/i-m-a-broken-mug-70f91b644217
https://medium.com/@evijit/new-year-resolutions-1a3ce87ff0ad
https://medium.com/january-rain/isaiah-60-1-2-89aff7d715f6
https://medium.com/@nealrobertsmookler/anxiety-5bccab23cd63
https://medium.com/@greggclunis/how-to-build-a-list-making-habit-and-why-you-should-in-2016-a344e93c0609
https://medium.com/confessions-of-a-poker-husband/confession-002-c1b2afe529fb
https://medium.com/come-as-you-are-blog/new-year-new-me-bollocks-6c1deb8fd359
https://medium.com/@mysarahsone/of-insecurities-anxieties-1eb914869ca5
https://medium.com/@mjackson/help-b56b1a06842
https://medium.com/the-coffeelicious/i-am-in-a-funk-d4f3d466e5c1
https://medium.com/@complexb21/it-s-ok-to-be-awkward-b559cf64e8f6
https://medium.com/@lucianadacostamarques/my-2016-resolutions-72777c6e4469
https://medium.com/@hardeekssha

https://medium.com/@frithahookway/on-anxiety-completely-unfiltered-6f577386de02
https://medium.com/@saracmd/bellletstalk-about-anxiety-d0452f30ae27
https://medium.com/@courtneyparaham/meeting-anxiety-for-the-thirtieth-time-ee8d66a1f2e8
https://medium.com/beard-n-butter/depression-a-disambiguation-1492a2cbbfc9
https://medium.com/@pfthurley/counting-my-depression-as-a-blessing-bellletstalk-2016-a1c4eadf7a9d
https://medium.com/@TrillSmith/dealing-with-anxiety-as-a-person-of-color-f04415c234f1
https://medium.com/@RoAnnaSylver/going-through-the-motions-joy-sadness-and-letting-it-out-e99dd5fd0944
https://medium.com/@guiltysquid/i-really-only-plan-for-ridiculous-disasters-the-cdc-has-me-covered-on-most-others-5061b823b986
https://medium.com/@Amysplus/thoughts-of-mine-213bf7819cf6
https://medium.com/@JoyaDerksen/the-thing-you-can-find-from-anxiety-psychological-services-calgary-98271bfd4779
https://medium.com/@hoydb/prioritizing-my-way-out-of-anxiety-60baadbc0441
https://medium.com/@slapnutz12

https://blog.nomad.pictures/going-digitally-dark-and-what-could-be-a-media-storytelling-win-for-mobile-operators-33d8d6170418
https://fityourself.club/panic-eb9abec57e3c
https://medium.com/@dtanga/how-my-anxiety-helps-me-overcome-my-obstacles-d1d22c0b687b
https://medium.com/@MiluRamalho/how-dishonesty-affects-you-48b775547960
https://medium.com/@TheQuietCreator/control-444a2f089051
https://medium.com/@matthijssteeneveld/the-anxiety-disorder-of-organizations-a605ae8785c3
https://medium.com/@BaffoeMan/a-letter-to-an-anxious-mind-from-an-unworried-one-fadd75040708
https://medium.com/@NilsvonKalm/jesus-was-not-a-nice-guy-59ed914110f6
https://medium.com/@DinithaBandara/thing-s-to-do-in-morning-before-going-to-school-b5e1addc953c
https://medium.com/@BizzyBee/anxiety-me-244befbb0ca2
https://medium.com/@nickgm/the-3-breath-reset-7a2ef6de621b
https://medium.com/@hannakcarter/my-anxiety-disorder-doesn-t-mean-i-m-crazy-efe4f2693721
https://medium.com/@philippaburgess/february-eighteen-put-forth-y

https://medium.com/come-as-you-are-blog/mind-over-matter-512a933c9595
https://medium.com/@bigrefill/float-like-a-butterfly-37b0e278d9cf
https://medium.com/@jamie.infanti/the-5-steps-to-overthinking-d659462c437e
https://medium.com/@michael_28419/seeing-and-feeling-abundance-creates-abundance-a04e40998457
https://medium.com/@happysciencemom/kids-and-caffeine-don-t-mix-68c829b4c02d
https://medium.com/@ya_burke/sadness-hopelessness-565dcf870dac
https://medium.com/@hopelg/dear-mr-prime-minister-justin-trudeau-1b4e7360fc6e
https://medium.com/@bethanylang/what-made-you-depressed-5daf830f9bb8
https://medium.com/@trinitariae/on-being-anxious-e16d8b0cd339
https://medium.com/@elissasanci/here-is-why-i-thank-my-anxiety-1e1216ec4417
https://medium.com/@michael_28419/fulfillment-dictionary-f14067288e98
https://medium.com/@fearlessjournal/choice-8eec1192d271
https://medium.com/@CPConrad/my-selfie-my-self-b4eb622b1387
https://medium.com/dumbfounded/waiting-is-painful-e90d4224161b
https://medium.com/@C

https://medium.com/@PuddlesGatherRain/fluoxetine-fun-times-b249108ddd51
https://medium.com/@japanjabara/asocial-experiment-my-love-hate-relationship-with-people-63e47eebf3fc
https://curiosityneverkilledthewriter.com/rumination-eac380e07027
https://medium.com/@aarohini/panacea-823e34d01ad4
https://medium.com/@paulaachew/as-freedom-is-a-breakfastfood-eb6587e5417b
https://medium.com/@tiisetsomaloma/the-man-who-fed-his-child-with-the-face-of-the-baby-mama-a-metaphor-for-creativity-81c3485f1667
https://medium.com/@summerlandaz/pain-and-panic-dog-biscuits-223a8ffc8d0
https://medium.com/@elizadelvie/first-person-31d1ac2fde12
https://medium.com/@MerelyGrace/stronger-than-we-know-4aa57780ce81
https://medium.com/@jourdenvsander/a-8222decd383
https://medium.com/@dominicabean13/how-to-deal-with-stage-fright-b0e5f047d65f
https://medium.com/@larnaud23/about-feels-buts-i-ams-and-ands-8896046fb244
https://medium.com/@glasshousegirls/mental-health-prevention-or-cure-4e7819897936
https://medium.com/@bra

...

https://medium.com/indian-thoughts/neurotica-part-3-7abe142cda90
https://medium.com/ryerson-university/the-weight-of-being-different-5efe9210d1f0
https://medium.com/mindfulness-with-a-v-sign/when-you-say-you-are-okay-but-youre-not-okay-6545d8923417
https://medium.com/researchfeatures/healthcare-delivery-soaring-to-new-heights-with-interactive-simulation-6d47c4371f9
https://medium.com/the-center-for-story-and-spirit/take-heart-the-power-of-poetry-in-troubled-times-7ce6a65c4def
https://medium.com/@On_MissionTV/the-kids-are-not-alright-teen-anxiety-and-depression-at-all-time-highs-8050e2dd5073
https://medium.com/@ResilientBella/meditation-for-beginners-20-practical-tips-for-understanding-the-mind-36258ad2e728
https://medium.com/@alexisassadi/anxiety-94caebf9ac6e
https://medium.com/@lancegpowelljr/pain-and-anxiety-in-vr-5e6c09ba6fc6
https://medium.com/@aloiolinho/sobre-a-ansiedade-3dba834cbdc9
https://medium.com/@thepenman/the-nightmare-diary-e6dfbf92c83b
https://medium.com/@jessicalazuka/

https://medium.com/@ewriedt001/how-do-we-fight-back-against-the-newest-attack-on-schools-cb98081f7f36
https://medium.com/@adminchang/self-sabatoge-95b912fa44a
https://medium.com/@harrisonmalone_/flying-699ccedfc2cf
https://medium.com/@vanessa.g/when-you-do-what-you-fear-most-then-you-can-do-anything-stephen-richards-6063cdbf6f64
https://medium.com/@geoffblades/how-to-deal-with-anxiety-and-use-it-to-your-advantage-4990a113b9e3
https://medium.com/obvi-were-the-ladies/my-feminist-ah-ha-moment-40054939f515
https://medium.com/@rafaelgmez/count-your-blessings-6a74b051827b
https://medium.com/@namelesscreativ/islands-ab4fa1b66f1e
https://medium.com/@andystitt829/my-name-is-andy-stitt-and-i-have-anxiety-f28bbe5652c9
https://medium.com/@anaees/a-full-share-of-serotonin-5e69d189fe9e
https://medium.com/thrive-global/how-to-achieve-a-lucid-dream-state-of-control-in-every-business-interaction-4075bacc4c2a
https://medium.com/@lead_time/an-antidote-to-anxiety-4fdca6460728
https://medium.com/@Dr.Dragos

https://medium.com/@joy.allison2014/befriending-the-monsters-2b3ae9e7d67b
https://medium.com/@patrick.se/how-to-stop-worrying-and-start-living-f70a0dab8f1d
https://medium.com/@mindyraf/my-fast-company-interview-d0ac2c4c3801
https://medium.com/rohan-mcleod-recommends/lavender-the-healing-candle-ebb68972bd31
https://medium.com/@robehill/a-lesson-to-calm-the-mind-2996bc86f0c9
https://medium.com/the-blessed-hub/decreasing-anxiety-and-fear-c804d8e287ab
https://medium.com/@welby84/psa-do-you-suffer-with-roommate-anxiety-disorder-cf40f6ea8781
https://medium.com/@psychologistdub/what-is-anxiety-what-are-anxiety-disorders-what-is-the-best-treatment-51c5581ae20f
https://medium.com/@Livable79/v%C3%ADctimas-de-nuestro-propio-infierno-eda977f8b6b9
https://medium.com/@johns893/social-anxiety-where-do-we-go-from-here-c77f9618da2f
https://medium.com/@j.radek/the-effects-of-social-anxiety-525be7d2e023
https://medium.com/bizcatalyst-360/should-you-live-for-your-resume-or-your-eulogy-f8b901f1e03c
https:/

https://medium.com/@savyleiser/and-4e7997f66672
https://medium.com/personal-growth/we-all-play-favorites-dont-we-6b9a84f712f
https://blog.heartsupport.com/generation-stress-and-how-we-can-overcome-worry-f189a4c66cfc
https://pulpnewsmag.com/a-ted-talk-an-end-goal-for-social-anxiety-d6b535dd8db2
https://medium.com/@drewannew/what-i-learned-from-132-hours-of-volunteering-in-the-tenderloin-f3dee91430c0
https://medium.com/@advocacypartner/would-you-still-hire-me-if-i-tell-you-i-once-suffered-from-depression-and-anxiety-e5f68d61052b
https://medium.com/college-anxiety/handling-change-ccfd60d60f41
https://medium.com/dawn-wiggins-therapy/why-do-people-lie-d63b00f0adae
https://medium.com/@anhere/growing-up-with-gad-57598ec3cb00
https://medium.com/@anjyrajy/here-is-what-you-need-to-remember-when-it-feels-like-your-anxiety-is-winning-2ec08af38c32
https://medium.com/everyday-mental-health/10-gift-ideas-for-your-loved-one-with-depression-and-anxiety-b4c5c7a9c8f6
https://medium.com/@ashley_montello/p

https://medium.com/@pku2319/as-it-begins-19237ade775e
https://medium.com/@Mindful.am/how-to-use-mindfulness-to-fight-holiday-depression-1ba10afd995
https://medium.com/the-maxximiliann/you-are-precious-you-are-important-you-are-valuable-89f95782c685
https://medium.com/@thepotbelliedpolitico/self-care-when-visiting-family-and-cookies-bebb67d4363e
https://42hire.com/40-days-of-unemployment-e134c49a6a42
https://42hire.com/whos-viewed-my-profile-e9e7c4e5f535
https://medium.com/@anxieticure/a-deadly-mistake-uncovered-on-treating-anxiety-and-how-to-avoid-it-147ee6c25ca4
https://medium.com/obvi-were-the-ladies/i-dont-really-like-my-birthday-really-i-don-t-ed81e722374f
https://medium.com/@HowardLeff/meditation-makes-me-nervous-f52af890f67a
https://medium.com/@johneclark/what-is-anxiety-or-dear-meatloaf-its-3-30-am-will-you-please-shut-up-d6a7adc52ff
https://medium.com/@elsievioletwaggoner/cant-keep-the-bitch-down-for-long-ed5eb8607821
https://medium.com/@gegoodsell/holiday-blues-21174eab1eed
ht

In [98]:
medium_df = pd.DataFrame(data, columns=['id', 'url', 'title', 'date', 'text'])

In [99]:
medium_df 

,id,url,title,date,text
0,1,https://medium.com/@elissaerin/27-minutes-of-u...,27 Minutes of Uber-Induced Anxiety,2016-01-01,Alternate title: The actual thoughts of a pers...
1,2,https://medium.com/@AliciaEdquist/thank-you-an...,"Thank you, Anxiety.",2016-01-01,I hate living with anxiety never thought I wou...
2,3,https://medium.com/@guiltysquid/i-m-a-broken-m...,I’m Like A Broken Mug,2016-01-01,Talking about your mental health issues is har...
3,4,https://medium.com/@evijit/new-year-resolution...,New Year Resolutions,2016-01-01,Today I am going to write about one of my most...
4,5,https://medium.com/january-rain/isaiah-60-1-2-...,Isaiah 60:1–2,2016-01-01,When I first thought about starting this blog ...
...,...,...,...,...,...
2752,2753,https://medium.com/invisible-illness/carry-you...,carry yourself with the air of normalcy,2016-12-31,breathe in count to five breathe out count to ...
2753,2754,https://medium.com/cry-mag/the-pain-is-all-tha...,The Pain Is All That Matters,2016-12-31,I don’t know where I’d be without this feeling...
2754,2755,https://medium.com/@saraweaver/words-7a591be384a2,Words,2016-12-31,"In the letter, I apologized and reconciled my ..."
2755,2756,https://medium.com/@TrevorKeatonPogue/in-this-...,Episode 4 Recap (The Resolutionists),2016-12-31,In this week’s episode of Anxiety Club Karl an...


In [100]:
medium_df = medium_df.drop(medium_df.columns[[0, 1, 3]], axis=1)

In [101]:
medium_df['year'] = year
medium_df

,title,text,year
0,27 Minutes of Uber-Induced Anxiety,Alternate title: The actual thoughts of a pers...,2016
1,"Thank you, Anxiety.",I hate living with anxiety never thought I wou...,2016
2,I’m Like A Broken Mug,Talking about your mental health issues is har...,2016
3,New Year Resolutions,Today I am going to write about one of my most...,2016
4,Isaiah 60:1–2,When I first thought about starting this blog ...,2016
...,...,...,...
2752,carry yourself with the air of normalcy,breathe in count to five breathe out count to ...,2016
2753,The Pain Is All That Matters,I don’t know where I’d be without this feeling...,2016
2754,Words,"In the letter, I apologized and reconciled my ...",2016
2755,Episode 4 Recap (The Resolutionists),In this week’s episode of Anxiety Club Karl an...,2016


In [102]:
fname = 'medium_text_{}.csv'.format(year)
medium_df.to_csv(fname, index=False)